In [1]:
import pandas as pd
import numpy as np
# from random import shuffle
from DiyLSTMmod import *
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [ ]:
#df = pd.read_excel('./dataset/Rice_MSC_Dataset.xlsx','Rice_MSC_Dataset')
df = pd.read_excel('./dataset/Rice_MSC_Dataset.xlsx')
#df = shuffle(df)
# df.reset_index(inplace=True,drop=True)

In [ ]:
df.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,ALLdaub4L,ALLdaub4a,ALLdaub4b,ALLdaub4Y,ALLdaub4Cb,ALLdaub4Cr,ALLdaub4XX,ALLdaub4YY,ALLdaub4ZZ,CLASS
0,7805,437.915,209.8215,48.0221,0.9735,99.6877,0.9775,7985,0.3547,4.3693,...,113.9924,65.0610,59.5989,104.8552,67.8779,63.0828,0.3673,0.3793,0.4733,Basmati
1,7503,340.757,138.3361,69.8417,0.8632,97.7400,0.9660,7767,0.6637,1.9807,...,105.7055,64.3685,62.2084,96.8375,65.5371,63.5832,0.3014,0.3144,0.3641,Arborio
2,5124,314.617,141.9803,46.5784,0.9447,80.7718,0.9721,5271,0.4760,3.0482,...,109.7155,62.6423,58.7439,100.2352,68.9753,59.8342,0.3233,0.3445,0.4448,Jasmine
3,7990,437.085,201.4386,51.2245,0.9671,100.8622,0.9659,8272,0.6274,3.9325,...,116.5405,64.9069,60.2562,107.2560,67.3298,63.2237,0.3880,0.4020,0.4904,Basmati
4,7433,342.893,140.3350,68.3927,0.8732,97.2830,0.9831,7561,0.6006,2.0519,...,107.7502,64.7071,61.3549,98.8704,66.2048,63.5378,0.3184,0.3303,0.3928,Arborio


'AREA,'PERIMETER','MAJOR_AXIS','MINOR_AXIS,'ECCENTRICITY','EQDIASQ','SOLIDITY','CONVEX_AREA','EXTENT','ASPECT_RATIO','ROUNDNESS','COMPACTNESS','SHAPEFACTOR_1','SHAPEFACTOR_2','SHAPEFACTOR_3' ,'SHAPEFACTOR_4','meanRR,'meanRG'

In [ ]:
data = df[['AREA','PERIMETER','MAJOR_AXIS','MINOR_AXIS','ECCENTRICITY','EQDIASQ','SOLIDITY','CONVEX_AREA','EXTENT','ASPECT_RATIO','ROUNDNESS','COMPACTNESS','SHAPEFACTOR_1','SHAPEFACTOR_2','SHAPEFACTOR_3' ,'SHAPEFACTOR_4','meanRR','meanRG']]

In [ ]:
class_lis =['Basmati', 'Arborio', 'Jasmine', 'Ipsala', 'Karacadag']
df_class = pd.DataFrame()
df_class['Class']=df['CLASS'].map(
{
    'Basmati':0,
    'Arborio':1, 
    'Jasmine':2, 
    'Ipsala':3,
    'Karacadag':4
})


In [ ]:
df_class

,Class
0,0
1,1
2,2
3,0
4,1
...,...
74995,1
74996,4
74997,1
74998,3


In [ ]:
LisClassNp = df_class['Class'].values
Y = np.array(LisClassNp)
df = df.drop(columns='CLASS')
X = df.values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

In [ ]:
#拆分训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.001, random_state=40)
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
#X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

X_train = tf.cast(X_train, dtype='float32')
#X_test = tf.cast(X_test, dtype='float32')
Y_train = tf.cast(Y_train, dtype='float32')
# Y_test = tf.cast(Y_test, dtype='float32')

In [ ]:
X_train.shape

TensorShape([74925, 1, 106])

In [ ]:
checkpoint_path = "./models_RICE/cp-{categorical_accuracy:.5f}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
train_acc = []
valid_acc = []
for i in range(10):
    t,v = k_fold(10, X_train, Y_train,100,128)
    train_acc.append(t)
    valid_acc.append(v)
print('训练集：',train_acc)
print('验证集:',valid_acc)

print('训练集平均，验证集平均：',ave_list(train_acc),ave_list(valid_acc))

Epoch 1/100


InvalidArgumentError: Matrix size-incompatible: In[0]: [128,106], In[1]: [7,128] [Op:MatMul]